# Probar los niveles de precisión de los modelos (LSTM, LASSO, TDA+LSTM[con valor k óptimo]) para distintas cantidades de samples

**Cargar las librerias**

In [41]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Masking
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

Cargar los datos

In [42]:

imagenes=np.load('datos/poll_imagv1.npy')
data = pd.read_csv('datos/train_poll_v1s1.csv')
val_glove = np.load('datos/encuestav1_gloVe_Values.npy', allow_pickle=True)
labels = data['ai']
labels = np.array(labels)
#Convert imagenes array from 3D to 2D
lol=[]
for i in range(len(imagenes)):
    lol.append(imagenes[i].flatten())
imagenes=np.array(lol)

### Crear dataframe para guardar los resultados

In [43]:
Resultado = pd.DataFrame(columns=['Tam', 'Acc LSTM', 'Acc TDA', 'Acc TDA+LSTM'])

### Ciclo para hacer las pruebas

In [44]:
for i in range(10):

    acc_LSTM=0
    acc_TDA=0
    acc_TDALSTM=0


    n=40+i*40
    w=4

    for i in range(10):
        #select a random sample of n samples
        randoms = random.sample(range(0, len(imagenes)),n)
        n_samples = n #with 63 the training set is 50 samples
        X = imagenes[randoms]#imagenes[:n_samples]
        y = labels[randoms]# Binary target

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Create a Logistic Regression model with L1 regularization
        alpha = 0.1  # Regularization strength 0.00001
        log_reg = LogisticRegression(penalty='l1', solver='liblinear', C=1/alpha)

        # Train the model
        log_reg.fit(X_train, y_train)

        # Make predictions
        y_train_pred = log_reg.predict(X_train)
        y_test_pred = log_reg.predict(X_test)

        # Evaluate the model
        train_accuracy = accuracy_score(y_train, y_train_pred)
        test_accuracy_TDA = accuracy_score(y_test, y_test_pred)


        #print(f'Training Accuracy: {train_accuracy:.4f}')
        #print(f'Testing Accuracy: {test_accuracy:.4f}')

        y_test_predLASSO_P = log_reg.predict_proba(X_test)
        LASSO = y_test_predLASSO_P[:,1]

        ####### LSTM ########

        # Generate some dummy data with variable-length sequences

        max_timesteps = 100  # Define the maximum length for padding

        #Select only the n first samples

        X = val_glove[randoms]
        y = labels[randoms]

        # Pad the sequences to have the same length
        X_padded = pad_sequences(X, maxlen=max_timesteps, dtype='float32', padding='post', truncating='post')
        X_train, X_val, y_train, y_val = train_test_split(X_padded, y, test_size=0.2, random_state=42)


        # Define the BiLSTM model
        model = Sequential()
        model.add(Masking(mask_value=0.0, input_shape=(max_timesteps, 300)))  # Mask padded values
        model.add(Bidirectional(LSTM(64, return_sequences=False)))
        model.add(Dense(1, activation='sigmoid'))

        # Compile the model
        model.compile(optimizer=Adam(), loss=BinaryCrossentropy(), metrics=['accuracy'])

        # Print the model summary
        #model.summary()

        # Train the model
        #model.fit(X_padded, y, epochs=5, batch_size=32, validation_split=0.2)
        model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_val, y_val),verbose=0)

        LSTM_P=model.predict(X_val,verbose=0)

        test_accuracy_LSTM=model.evaluate(X_val, y_val)[1]



        # Combinar ambas predicciones
        pred_final=[]
        for i in range(len(LSTM_P)):
            pred_final.append((LASSO[i]+LSTM_P[i]*w)/(1+w))
        pred_final_label=[]
        for i in pred_final:
            if i>0.5:
                pred_final_label.append(1)
            else:
                pred_final_label.append(0)
        #count the number of correct predictions
        counter=0
        for i in range(len(pred_final_label)):
            if pred_final_label[i]==y_test[i]:
                pass
                counter+=1
                #print('Correcto')
            elif pred_final_label[i]!=y_test[i]:
                #print('Falso')
                pass

        #calculate the accuracy

        test_accuracy_TDALSTM=counter/len(pred_final_label)

        acc_LSTM+=test_accuracy_LSTM
        acc_TDA+=test_accuracy_TDA
        acc_TDALSTM+=test_accuracy_TDALSTM

    acc_LSTM=acc_LSTM/10
    acc_TDA=acc_TDA/10
    acc_TDALSTM=acc_TDALSTM/10
    res=[n,acc_LSTM, acc_TDA, acc_TDALSTM]
    Resultado.loc[i]=res
    print(res)
    print('------------------------------------')

d:\miniconda\envs\gcwtdaENV\lib\site-packages\keras\src\layers\core\masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5000 - loss: 0.6744


d:\miniconda\envs\gcwtdaENV\lib\site-packages\keras\src\layers\core\masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3750 - loss: 0.6675


d:\miniconda\envs\gcwtdaENV\lib\site-packages\keras\src\layers\core\masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9336 - loss: 0.3315


d:\miniconda\envs\gcwtdaENV\lib\site-packages\keras\src\layers\core\masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.9797 - loss: 0.2082


d:\miniconda\envs\gcwtdaENV\lib\site-packages\keras\src\layers\core\masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.9656 - loss: 0.3055
[400, 0.9425000011920929, 0.82875, 0.9475]
------------------------------------


In [45]:
# Save resultados
Resultado.to_csv('Resultados.csv', index=False)